In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Respiradores;
CREATE TABLE Respiradores (
    Data VARCHAR(10),
    Fornecedor VARCHAR(100),
    Destino VARCHAR(100),
    EstadoMunicipio VARCHAR(100),
    Tipo VARCHAR(100),
    Quantidade INTEGER,
    Valor FLOAT,
    UF VARCHAR(2),
    DataEntrega VARCHAR(10),
) AS SELECT
    DATA,
    FORNECEDOR,
    DESTINO,
    ESTADOMUNICIPIO,
    TIPO,
    QUANTIDADE,
    VALOR,
    UF,
    DATADEENTREGA
FROM CSVREAD('distribuicao_respiradores_certo.csv');

In [3]:
DROP TABLE IF EXISTS VerbaEstado;
CREATE VIEW VerbaEstado AS
    SELECT R.UF, SUM(R.Valor) as TotalRecebido
    FROM Respiradores as R
    GROUP BY R.UF;

SELECT *
FROM VerbaEstado as VE
GROUP BY VE.UF
ORDER BY VE.TotalRecebido DESC;

In [34]:
DROP TABLE IF EXISTS VerbaRegiao;
CREATE TABLE VerbaRegiao (
    Regiao VARCHAR(20),
    Verba FLOAT,
);

INSERT INTO VerbaRegiao VALUES ('Sudeste',(SELECT SUM(VE.TotalRecebido)
                                            FROM VerbaEstado as VE
                                            WHERE VE.UF = 'SP' OR VE.UF = 'RJ' OR VE.UF = 'ES' OR VE.UF = 'MG'));
                                            
INSERT INTO VerbaRegiao VALUES ('Norte',(SELECT SUM(VE.TotalRecebido)
                                            FROM VerbaEstado as VE
                                            WHERE VE.UF = 'AM' OR VE.UF = 'RR' OR VE.UF = 'AP' OR VE.UF = 'PA' OR VE.UF = 'TO' OR VE.UF = 'RO' OR VE.UF = 'AC'));
                                            
INSERT INTO VerbaRegiao VALUES ('Nordeste',(SELECT SUM(VE.TotalRecebido)
                                            FROM VerbaEstado as VE
                                            WHERE VE.UF = 'MA' OR VE.UF = 'PI' OR VE.UF = 'CE' OR VE.UF = 'RN' OR VE.UF = 'PE' OR VE.UF = 'PB' OR VE.UF = 'SE' OR VE.UF = 'AL' OR VE.UF = 'BA'));
                
INSERT INTO VerbaRegiao VALUES ('CentroOeste',(SELECT SUM(VE.TotalRecebido)
                                            FROM VerbaEstado as VE
                                            WHERE VE.UF = 'MT' OR VE.UF = 'MS' OR VE.UF = 'GO' OR VE.UF = 'DF'));
                                            
INSERT INTO VerbaRegiao VALUES ('Sul',(SELECT SUM(VE.TotalRecebido)
                                            FROM VerbaEstado as VE
                                            WHERE VE.UF = 'PR' OR VE.UF = 'RS' OR VE.UF = 'SC'));

SELECT * 
FROM VerbaRegiao as VR
GROUP BY VR.Regiao
ORDER BY VR.Verba DESC;

In [39]:
DROP TABLE IF EXISTS CasosRegiao;
DROP TABLE IF EXISTS CasosEstado;

CREATE TABLE CasosEstado (
    Regiao VARCHAR(20),
    Fornecedor VARCHAR(2),
    CodUF INTEGER,
    Data VARCHAR(100),
    SemanaEPI INTEGER,
    Populacao INTEGER,
    CasosAcumulado INTEGER,
    CasosNovos INTEGER,
    ObitosAcumulados INTEGER,
    ObitosNovos INTEGER,
) AS SELECT
    regiao,
    estado,
    coduf,
    data,
    semanaEpi,
    populacaoTCU2019,
    casosAcumulado,
    casosNovos,
    obitosAcumulado,
    obitosNovos
FROM CSVREAD('casos_por_estado.csv');

SELECT *
FROM CasosEstado;

In [40]:
DROP TABLE IF EXISTS CasosRegiao;
CREATE VIEW CasosRegiao AS
    SELECT CE.Regiao, SUM(CE.CasosAcumulado) AS TotalCasos,SUM(CE.ObitosAcumulados) AS TotalObitos
    FROM CasosEstado AS CE
    GROUP BY CE.Regiao;

SELECT * 
FROM CasosRegiao;

In [47]:
SELECT CR.Regiao, CR.TotalObitos, VR.Verba, VR.Verba/CR.TotalObitos AS VerbaPorObito, CAST((1.0*100*CR.TotalObitos/CR.TotalCasos) AS FLOAT) AS Mortalidade
FROM CasosRegiao AS CR, VerbaRegiao AS VR
WHERE CR.Regiao=VR.Regiao
GROUP BY CR.Regiao
ORDER BY Mortalidade DESC;


In [51]:
SELECT CE.Fornecedor AS Estado, CE.ObitosAcumulados AS Obitos, CE.CasosAcumulado AS Casos, VE.TotalRecebido AS Verba, VE.TotalRecebido/CE.ObitosAcumulados AS VerbaPorObito, 
        CAST(1.0*100*CE.ObitosAcumulados/CE.CasosAcumulado AS FLOAT) AS Mortalidade
FROM CasosEstado AS CE, VerbaEstado AS VE
WHERE CE.Fornecedor=VE.UF
GROUP BY VE.UF
ORDER BY Mortalidade DESC;

org.h2.jdbc.JdbcSQLException:  Syntax error in SQL statement "JAVASCRIPT[*](JS_DOWNLOAD) "; SQL statement